In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [4]:
X = pd.read_csv('./data/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')

In [4]:
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [5]:
X.head()

essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   domain1_score  
0              8  
1              9  
2              7  
3             10  
4              8

In [6]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

# Preprocessing

In [ ]:
#download the stopwords if you have not downloaded
nltk.download('stopwords')

In [8]:
#removing the extra characters other than alphabets and stopwords and tokenizing the words
def essay_to_wordlist(essay_v):
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    return (words)

#Tokenize the senstences and call essay_to_wordlist() for word tokenization.
def essay_to_sentences(essay_v):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence))
    return sentences


In [9]:
#Feature vector is made from the words list of an essay.
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

#Word vectors are generated for Word2Vec model
def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [10]:
#the dataset is split to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [11]:
X_train.shape

(10380, 4)

In [12]:
X_test.shape

(2596, 4)

In [13]:
train_essays = X_train['essay']
test_essays = X_test['essay']

In [14]:
sentences = []
# Obtaining all sentences from the training essays.
for essay in train_essays:
    sentences += essay_to_sentences(essay)

In [15]:
sentences

[['essay', 'author', 'talking', 'journey'],
 ['shows',
  'give',
  'ends',
  'almost',
  'way',
  'began',
  'shows',
  'people',
  'stay',
  'attached',
  'past',
  'reasons',
  'believe',
  'author',
  'concludes',
  'story',
  'paragraph'],
 ['shows',
  'people',
  'give',
  'even',
  'though',
  'failed',
  'test',
  'try',
  'later',
  'pass'],
 ['also',
  'story',
  'ends',
  'way',
  'begins',
  'taking',
  'test',
  'getting',
  'ready',
  'good',
  'way',
  'writing'],
 ['one',
  'last',
  'thing',
  'shows',
  'people',
  'become',
  'attached',
  'past',
  'look',
  'one',
  'thing',
  'reminds',
  'month',
  'fall',
  'apart',
  'takes',
  'test',
  'see',
  'look',
  'familiar',
  'flowers',
  'without',
  'falling',
  'apart'],
 ['story',
  'winter',
  'hibiscus',
  'minfong',
  'ho',
  'think',
  'ending',
  'paragraph',
  'show',
  'saeng',
  'given'],
 ['says', 'come', 'back', 'take', 'test', 'talking', 'hibiscus', 'geese'],
 ['things', 'remind', 'home'],
 ['reminders'

In [16]:
#build the vectorizer with maximum featuroes of 300
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, 
                 sample = downsampling)

In [17]:
#save the vectorizer in .bin file
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

In [18]:
#get the training vectors
clean_train_essays = []
for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
 
#get the testing vectors
clean_test_essays = []
for essay_v in test_essays:
    clean_test_essays.append(essay_to_wordlist( essay_v))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
#convert the vectors to numpy array
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

C:\Users\Sai Nidhi\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [31]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

Using TensorFlow backend.


In [32]:
model = Sequential()
model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
model.add(LSTM(64, recurrent_dropout=0.4))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [ ]:
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

In [34]:
model.fit(trainDataVecs, y_train, batch_size=64, epochs=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/2
10380/10380 [==============================] - 7s 660us/step - loss: 50.8579 - mean_absolute_error: 3.5178
Epoch 2/2
10380/10380 [==============================] - 4s 379us/step - loss: 28.4512 - mean_absolute_error: 2.5056


In [19]:
testDataVecs.shape

(2596, 1, 300)

In [37]:
y_pred = model.predict(testDataVecs)

In [38]:
model.save('final_lstm.h5')

In [43]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test,y_pred)
accuracy

0.7300429680972389